In [1]:
!pip install pandas
!pip install cplex
!pip install datetime

In [1]:
# __________________________________ ALL REQUIRED IMPORTS __________________________________

import sqlite3
import pandas as pd
from datetime import datetime

In [2]:
# __________________________________ LOADING DATA INTO A DATABASE __________________________________

# URLs for data sources
DATA_URLS = {

    "inventory": "https://raw.githubusercontent.com/team79interiit/qcc_data_files/main/inv.csv",
    "schedule": "https://raw.githubusercontent.com/team79interiit/qcc_data_files/main/sch.csv",
    "pnr_booking": "https://raw.githubusercontent.com/team79interiit/qcc_data_files/main/pnrb.csv",
    "pnr_passenger": "https://raw.githubusercontent.com/team79interiit/qcc_data_files/main/pnrp.csv",
    "pnr_booking_template": "https://raw.githubusercontent.com/team79interiit/qcc_data_files/main/booking_template.csv",

}

DATABASE = "data.db"

In [3]:
# Function to load data from URLs into dataframes
def load_data(data_urls=dict()):

    data_frames = {}

    for key, url in data_urls.items():

        data_frames[key] = pd.read_csv(url)

    return data_frames

In [4]:
# Function to add a DataFrame as a table to a SQLite database
def add_table_to_database(dataframe, tablename=''):

    try:

        # Establish connection to SQLite database
        connection = sqlite3.connect(DATABASE)

        # Add DataFrame as a table to the database
        dataframe.to_sql(tablename, connection, if_exists='fail', index=False)
        print(f"Table '{tablename}' added to the database.")

    except Exception as e:

        # Handle case where table already exists in the database
        print(f"{e}")

    finally:

        # Ensure the connection is closed after operations
        connection.commit()
        connection.close()

In [5]:
# Load data frames from URLs
data_frames = load_data(DATA_URLS)

# Select specific tables to add to the database
tables_to_add = {

    'inv': data_frames['inventory'],
    'pnrb': data_frames['pnr_booking'],
    'pnrp': data_frames['pnr_passenger'],
}

# Add selected tables to the database
for table_name, df in tables_to_add.items():
  add_table_to_database(df, table_name)

URLError: <urlopen error [Errno 110] Connection timed out>

In [ ]:
#  ___________________ WE ARE ASSUMING THAT THE FOLLOWING MINIMAL INFO IS GIVEN ON CHANGES IN SCHEDULE ______________

# Define constants for types of changes
CANCELLED = "Cancelled"
DELAYED = "Delayed"

# List of available types of changes
type_of_changes = [CANCELLED, DELAYED]

# Information about changes in flight schedules
changes = [
    {
        "Flight Number": 2504,
        "Departure Date": '4/3/2024',
        "Type of change": CANCELLED,
    },
    {
        "Flight Number": 3723,
        "Departure Date": "04/07/2024",
        "Type of change": DELAYED,
        "New Departure Date Time": "2024-05-05 11:52:00",
        "New Arrival Date Time": "2024-05-05 20:30:00",
    }
]

# Since the date & time in database are inconsistent we are using a global data structre to keep track of format while writing code
FORMATS = {
    "inv":[{
        "DepartureDate" : "%m/%d/%Y",
        "DepartureDateTime": "%Y-%m-%d %H:%M:%S",
        "ArrivalDateTime": "%Y-%m-%d %H:%M:%S",
    }, True], # True indicates that there is 0 infront of single digit numbers in dates

    "pnrb":[{
        "DEP_DT" : "%m/%d/%Y",
        "DEP_DTML" : "%m/%d/%Y %H:%M",
        "ARR_DTML": "%m/%d/%Y %H:%M",
        "DEP_DTMZ": "%m/%d/%Y %H:%M",
        "ARR_DTMZ": "%m/%d/%Y %H:%M",
    }, False], # False indicates that there is no 0 infront of single digit numbers in dates
}

In [ ]:
def search_for_affected_flight_inv(flight_number, departure_date):

  connection = sqlite3.connect(DATABASE)
  cursor = connection.cursor()

  sql_search_query = '''
  SELECT * FROM inv
  WHERE FlightNumber = ? and DepartureDate = ?;
  '''

  # The following code will make sure the departure_date is in correct format & also adds 0's if single digit date
  try:
    departure_date = datetime.strptime(departure_date, "%m/%d/%Y").strftime("%m/%d/%Y")
  except Exception as e:
    print("departure date is not in %m/%d/%Y format")

  cursor.execute(sql_search_query,(flight_number, departure_date))
  results = cursor.fetchall()

  connection.commit()
  connection.close()
  return results

In [ ]:
def search_for_affected_pnrb(flight_number, departure_date):

    connection = sqlite3.connect(DATABASE)
    cursor = connection.cursor()

    sql_search_query = '''
    SELECT * FROM pnrb
    WHERE FLT_NUM = ? AND DEP_DT = ?;
    '''

    # Attempt to convert departure_date to the correct format
    try:
        departure_date = datetime.strptime(departure_date, "%m/%d/%Y").strftime("%m/%d/%Y")
    except Exception as e:
        print("Departure date is not in the format %m/%d/%Y.")

    # Remove leading zeros from single-digit days or months
    departure_date_parts = departure_date.split('/')
    departure_date = '/'.join(str(int(part)) for part in departure_date_parts)

    cursor.execute(sql_search_query, (flight_number, departure_date))
    results = cursor.fetchall()

    connection.commit()
    connection.close()
    return results

In [ ]:
def update_inv_with_newdatetime (new_dept_time='', new_arr_time='', inv_id=''):

    connection = sqlite3.connect(DATABASE)
    cursor = connection.cursor()

    # Using parameterized query to avoid SQL injection
    sql_update_query = '''
    UPDATE inv
    SET "DepartureDateTime" = ?, "ArrivalDateTime" = ?
    WHERE InventoryId = ? '''

    # Attempt to convert departure_date to the correct format
    try:

        new_dept_time = datetime.strptime(new_dept_time, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S")

    except Exception as e:

        print("new_dept_time is not in the format %Y-%m-%d %H:%M:%S")

    # Attempt to convert departure_date to the correct format
    try:

        new_arr_time = datetime.strptime(new_arr_time, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S")

    except Exception as e:

        print("new_arr_time is not in the format %Y-%m-%d %H:%M:%S")

    # Tuple of values to be substituted into the query
    values = (new_dept_time, new_arr_time, inv_id)

    cursor.execute(sql_update_query, values)
    connection.commit()
    connection.close()

In [ ]:
def add_affected_passengers_to_database(inventory_id, flight_number, departure_date):

  pnr_booking_template_df = pd.read_csv(DATA_URLS["pnr_booking_template"])
  connection = sqlite3.connect(DATABASE)
  cursor = connection.cursor()

  # Identify the affected passengers and pass it on to post process
  # Step 1 : Create a table with Inventory Id of the affected flight as table name
  try:

    table_name = inventory_id.replace('-','_')
    pnr_booking_template_df.to_sql(table_name, connection, if_exists='replace', index=False)

    # Step 2 : Add the affected passengers to the table
    affected_pnrs = search_for_affected_pnrb(flight_number, departure_date)

    for pnr in affected_pnrs:

      insert_data_query = f'''
      INSERT INTO  {table_name}
      VALUES {pnr}
      '''
      cursor.execute(insert_data_query)

  except Exception as e:

    print(f"HURRAY! -- The passengers of the flight on this Inventory entry are already identified | {e} ")

  connection.commit()
  connection.close()
  return

In [ ]:
affected_flights = dict()

for change in changes:

  flight_number = change["Flight Number"]
  departure_date = change["Departure Date"]
  type_of_change = change["Type of change"]

  search_result = search_for_affected_flight_inv(flight_number, departure_date)

  if search_result == []:

    print(f" OOPS! ----- No Inventory entry exists for the given change : {change} ------ ")
    continue

  search_result = search_result[0]

  # Preparing to handling the change in schedule
  # dictionary to pass the information to the quantum solver

  inventory_id = search_result[0]

  if type_of_change in type_of_changes:

    add_affected_passengers_to_database(inventory_id, flight_number, departure_date)

    if type_of_change == CANCELLED:

      affected_flights[inventory_id] = type_of_change

    elif type_of_change == DELAYED:

      affected_flights[inventory_id] = type_of_change

      new_departure_date_time = change["New Departure Date Time"]
      new_arrival_date_time = change["New Arrival Date Time"]

      update_inv_with_newdatetime(new_departure_date_time, new_arrival_date_time, inventory_id)

    else:

      print(f"----- Unidentified type of change : {type_of_change} ------")

# print(affected_flights)

In [ ]:
# __________________________________ RULE ENGINE __________________________________

MAX_ETD_FROM_IMPACTED_FLIGHT_IN_HOURS = 72
MAX_CONNECTION_TIME = 12
MIN_CONNECTION_TIME = 1

# This reference table can be used for identifying the cabin for each class of service
CABIN_CLASS_MAP = {
    "J": ["A", "D", "J"],
    "F": ["F", "B"],
    "Y": ["S", "V", "W", "Z", "O", "S", "T", "U", "M", "N", "Y"]
}

# This will be Used when Original class is not available to rebook
CLASS_TO_CLASS_MAP = {
    "A": ["F", "S"],
    "S": ["F", "A"],
    "F": ["S", "A"],
    "J": ["O", "C", "I"],
    "C": ["O", "J", "I"],
    "I": ["J", "C", "O"],
    "O": ["J", "C", "I"],
    "Y": ["B", "P", "Z"],
    "B": ["Y", "P", "Z"],
}

# 1. This is only applicable if Downgrade is on.
# 2. If Downgrade to Cabin J/F then refer to Downgrade class respectively
DOWNGRADE_CLASS_MAP = {
    "J": "Y",
    "F": "Y",
}

# 1. This is only applicable if Upgrade is enabled.
# 2. If Upgraded to Cabin F/J then refer to Upgrade class respectively
UPGRADE_CLASS_MAP = {
    "Y": "F",
    "F": "J",
}

class PNR:
    def __init__(self):

        self.Recloc = None
        self.TYPE = None
        self.SSR = None
        self.Cabin = None
        self.Class = None
        self.Number_Of_Down_lineConnections = 0
        self.PaidServices = None
        self.Booked_As = None
        self.Number_of_PAX = None
        self.Loyalty = None
        self.ForceKickOut = None

    def Score(self):

        Special_Services = ["INFT", "WHCR", "WCHS", "WCHC", "LANG", "CHILD", "EXST", "BLND", "DEAF"]
        total_score = None
        self.ForceKickOut = True

        # SSR Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.SSR in Special_Services:
            total_score += 200

        # Cabin Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.Cabin:
            cabin_score_mapping = {
                "J": 2000,
                "F": 1700,
                "Y": 1500
            }
            total_score += cabin_score_mapping.get(self.Cabin, 0)

        # Class Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.Class:
            class_score_mapping = {
                "A": 1000,
                "C": 700,
                "K": 500,
            }
            total_score += class_score_mapping.get(self.Class, (0, 0))[0]

        # Connection Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.Number_Of_Down_lineConnections is not None:
            total_score += 100 * self.Number_Of_Down_lineConnections

        # Paid Service Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.PaidServices == "Yes":
            self.ForceKickOut = False
            total_score += 200

        # Booking-Type Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.Booked_As == "Group":
            self.ForceKickOut = False
            total_score += 500

        # No of PAX Score
        if self.TYPE == "PNR.INDIVIDUAL" and self.Number_of_PAX is not None:
            self.ForceKickOut = False
            total_score += 50 * self.Number_of_PAX


In [ ]:
def get_affected_pnrb (table_name):

  connection = sqlite3.connect(DATABASE)
  cursor = connection.cursor()

  sql_search_query = f'''
  SELECT * FROM {table_name}
  '''

  cursor.execute(sql_search_query)
  results = cursor.fetchall()

  connection.commit()
  connection.close()
  return results

In [ ]:
flights = list(affected_flights.keys())
for flight in flights:
  flight_data = get_affected_pnrb(flight.replace('-', '_'))
  print(flight_data)

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from qiskit_algorithms import QAOA
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
import copy
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import MinimumEigenOptimizer, WarmStartQAOAOptimizer, CplexOptimizer
from qiskit.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils.algorithm_globals import algorithm_globals

We are creating a QUBO instance of a Quadratic Program and solving it using Warm Start QAOA.

Quantum Solver Class does the following:
1. takes as an input the affected flight's inventory id
2. maintains an list of relevant flights
3. Creates the appropriate Quadratic Program
4. Converts it into QUBO instance
5. Solve it using Warm Start QAOA (we use classical optimiser as either Cplex or COBYLA)
6. Return the list of alternate flights 

What the user needs to do:
1. Setup an object of the QuantumSolver
2. Call solve() method

Flow of the functions:
1. __init__
2. __preProcess

1. solve()
2. quantumSolve()
3. __run()
4. QAOA_algo()
5. __postProcess()

In [3]:
class QuantumSolver:
    """
    Create Object of this class with required arguments .
    Call Solve method to solve the problem.
    """

    df = None  # INV.csv
    highval = 1000000  # used for incresing the weight of matrix
    inv_id: str  # stores the affected flight inventory id
    flight = None  # affected flight row
    FullList:dict  # Stores InventoryId of all the dirsrupted flights
    # =========
    # matrices to be used in Quadratic Program
    MatFlightTime: np.ndarray # Stores flight time of xi th flight. This will be also used to remove already got solutions.
    MatArrArp    : np.ndarray # 1 if ith flight is departure airport = impacted flight departure airport
    MatDepArp    : np.ndarray # 1 if ith flight is Arrival airport = impacted flight Arrival airport
    MatNeigh     : np.ndarray # 1 if ith flight have a flight whose arrivalAirport == departureAirport else 0
    MatGndDelay  : np.ndarray # stores the ground delay between ith and jth flight.
    MatTltDelay  : np.ndarray # stores total delay if this node is the last node(This remains zero for all flights whose arrival  airport is not = arrival airport of impacted flight)
    # ========

    def __init__(self, inv_id, ImpactedFlights:dict, dataFrame):
        """
            Input : 
                inv_id:Corresponding inventory id of affected flight from the provided datafile
                ImpactedFlights: Dictionary of all the impacted flights
                dataFrame: Input dataFrame To take the flights from

            Creates a list of relevant flights, initializes matrices, and calls the QuantumSolve() function
        """

        self.df = dataFrame

        self.inv_id = inv_id
        self.FullList=ImpactedFlights

        self.lst = self.__preProcess()  # list of relevant flights
        if len(self.lst) == 0:  # no relevant flights present
            raise Exception('No alternate flights')
        print(len(self.lst), "Feasible Flights\n")
        
        self.length = len(self.lst)
        self.length=min(self.length,10)


        ##===============
        # #Temporary ::saves all the selected flights to be used by the solver . Turn This on to see all the selected flights.

        # file=open("Selected Flights","w")
        # for i in self.lst[:self.length]:
        #     file.write(str(i)+"\n\n")
        # file.close()
        ##===============

        # Initializing above mentioned flights
        self.MatFlightTime = np.zeros((self.length, self.length))
        self.MatNeigh, self.MatGndDelay= np.zeros_like(self.MatFlightTime), np.zeros_like(self.MatFlightTime)
        self.MatArrArp, self.MatDepArp, self.MatTltDelay=np.zeros((self.length,)),np.zeros((self.length,)),np.zeros((self.length,))

        print("\nAffected Flight\n")
        print(self.flight)


    def solve(self):
        """
        Returns the list of all the alternate solutions in order of best to worse.
        """
        startTime = dt.datetime.now()
        ans = self.quantumSolve()  # invoke the quantum solve function
        TimeTaken=(dt.datetime.now() - startTime)
        print("TimeTaken:",TimeTaken)

        return ans


    def __diff(self, datetime1,datetime2):
        """
            returns difference in time of date2-date1
        """
        dt1 = dt.datetime.strptime(datetime1, "%m/%d/%Y %H:%M:%S")
        dt2 = dt.datetime.strptime(datetime2, "%m/%d/%Y %H:%M:%S")
        difference = dt2 - dt1
        return int(difference.total_seconds() // 60)
    

    def __preProcess(self) -> list:
        """
            Creates a list of relevant flights which have sensible departure and arrival times
        """

        # refer to above rule set to check which rule is being implemented
        index = self.df.loc[self.df["InventoryId"] == self.inv_id].index[0]  # get the index from the dataset
        self.flight = self.df.loc[index]  # affected flight
        list_of_feasible_flights = []
        impactedFlightStatus=self.FullList.get(self.flight["InventoryId"])
        currFlTime=self.__diff(datetime1=self.flight["DepartureDateTime"],datetime2=self.flight["ArrivalDateTime"])   # store flight time of impacted flight.
        arrivalsAt=dict()
        departsAt=dict()

        for i in range(len(self.df)):
            data = self.df.loc[i]
            if index == i:
                continue
            fromlst=self.FullList.get(data["InventoryId"])
            if fromlst:
                if fromlst=="Cancelled":  # If Flight is cancelled then do not add it. rule 8
                    continue
            if data["ArrivalAirport"] == self.flight["DepartureAirport"] or data["DepartureAirport"]==self.flight["ArrivalAirport"]:  # rule 5,6
                continue
            timebtwArr_Curdep = self.__diff(datetime1=self.flight["DepartureDateTime"], datetime2=data["ArrivalDateTime"])
            timebtwdep_CurrDep = self.__diff(datetime1=self.flight["DepartureDateTime"], datetime2=data["DepartureDateTime"])
            flightTime=self.__diff(datetime1=data["DepartureDateTime"], date2=data["ArrivalDateTime"])
            
            if impactedFlightStatus != "Cancelled" and self.__diff(datetime1=self.flight["ArrivalDateTime"], date2=data["ArrivalDateTime"]) > 0: # rule 9 If this flight is not cancelled then do not consider any flight taking more time than this.
                continue    
            if timebtwArr_Curdep < 60 or timebtwdep_CurrDep > 72 * 60:   # rule 1,2,3  # following the Minimum Ground Time and Maximum Connecting Time
                continue
            elif flightTime > currFlTime and data["DepartureAirport"] != self.flight["DepartureAirport"] \
                    and data["ArrivalAirport"] != self.flight["ArrivalAirport"]:  # rule 4
                continue
            else:
                list_of_feasible_flights.append(data)
                if data["ArrivalAirport"] in arrivalsAt.keys():
                    arrivalsAt[data["ArrivalAirport"]]+=1
                else:
                    arrivalsAt[data["ArrivalAirport"]]=1
                if data["DepartureAirport"] in departsAt.keys():
                    departsAt[data["DepartureAirport"]]+=1
                else:
                    departsAt[data["DepartureAirport"]]=1

        for fl in list_of_feasible_flights:  # Removes all those flights for which no other flights preceeds them or no other flight succeed them
            if (fl["DepartureAirport"] not in departsAt.keys() or fl["ArrivalAirport"] not in arrivalsAt.keys()) and fl["DepartureAirport"] != self.flight["DepartureAirport"] and \
                fl["ArrivalAirport"]!= self.flight["ArrivalAirport"]:
                    list_of_feasible_flights.remove(fl)

        return list_of_feasible_flights

    def __run(self):
        """
            Creates the Matrices required for the Quadratic Program
        """
        ConsideredFlights = dict()  # to keep a tally of unique flights

        # Nodes
        for i in range(self.length):
            data = self.lst[i]  # a single flight

            if data["InventoryId"] not in ConsideredFlights:
                ConsideredFlights[data["InventoryId"]] = (
                    data["DepartureAirport"],
                    data["ArrivalAirport"],
                    data["DepartureDateTime"],
                    data["ArrivalDateTime"],
                )

                # Q Matrix measures the total time taken by the flight (Arr - Dep of the flight)
                self.MatFlightTime[i, i] = self.__diff(data["DepartureDateTime"], data["ArrivalDateTime"]) * 100

                # putting values to the matrices as defined above
                if data["DepartureAirport"] == self.flight["DepartureAirport"]:
                    self.MatArrArp[i] = 1
                    self.MatNeigh[i, i] = 1
                else:
                    self.MatArrArp[i] = 0
                if data["ArrivalAirport"] == self.flight["ArrivalAirport"]:
                    self.MatDepArp[i] = 1
                    self.MatTltDelay[i]=int(self.__diff(self.flight["DepartureDateTime"],data["ArrivalDateTime"]) * 100) 
                else:
                    self.MatDepArp[i]= 0

                # Edges
                for j in range(self.length):
                    fl2 = self.lst[j]
                    if self.MatNeigh[i, i] == 1 and i != j:
                        self.MatGndDelay[i, j] = 0
                    else:
                        if data["DepartureAirport"] == fl2["ArrivalAirport"]:
                            self.MatNeigh[i, j] = 1
                            ti = int(self.__diff(fl2["ArrivalDateTime"], data["DepartureDateTime"]) * 100)  # minutes(fl2["DepartureTime"]) - minutes(data["ArrivalTime"])
                            if ti < 6000 or ti > 72000:
                                self.MatGndDelay[i, j] = self.highval
                                self.MatNeigh[i,j]=0
                            else:
                                self.MatGndDelay[i, j] = ti
                        else:
                            self.MatGndDelay[i, j] = 0


        self.MatGndDelay = self.MatGndDelay.astype(int)


    def QAOA_algo(self) -> list:
        """
        Main Function of the Class
        Quadratic Program : 1. Objective Function - MatFlightTime+MatGndDelay + MatTltDelay(linear)
                            2. Linear Constraints - MatArrArp & MatDepArp
                            3. Quadratic Constraints - MatNeigh
        Why QUBO - Each flight has two options - Either it can be in the path or not
        :return: list of alternate flights as
        """

        qp = QuadraticProgram("flights")
        F = self.MatFlightTime + self.MatGndDelay # quadratic form matrix
        F = F.astype(int)

        qp.minimize(linear=self.MatTltDelay,quadratic=F)  # matrices fed into the quadratic program
        qp.binary_var_list(self.length)
        qp.linear_constraint(np.ones(self.length, ), ">", 1)  # alteast 1 flights must be selected
        # qp.linear_constraint(np.ones(self.length, ), "<", 5)  # Maximum 5 flights must be selected
        qp.linear_constraint(self.MatArrArp, "=", 1)          # There must be only and necessarily 1 flight with departureAirport = DepartureAirport of impacted flight
        qp.linear_constraint(self.MatDepArp, "=", 1)          # There must be only and necessarily 1 flight with ArrivalAirport = ArrivalAirport of impacted flight
        for i in range(self.length):                          # Applies the constraint that if a flight is on(1) then there must atleast 1 neighbouring flight that is on(1)
            linear=copy.deepcopy(self.MatNeigh[i,:])
            linear[i]-=1
            qp.linear_constraint(linear,">",0)
            
        # qp = QuadraticProgramToQubo().convert(qp)
        print("Variables : ",qp.get_num_binary_vars)

        # Classical Solver 
        # start1 = dt.datetime.now()
        # ws_qaoa_result=(CplexOptimizer().solve(qp))
        # print(dt.datetime.now() - start1)
        # print(qp.prettyprint())

        # Quadratic  Solver

        # qubitOp, offset = qp.to_ising()  # conversion into Ising Problem

        # start = dt.datetime.now()

        algorithm_globals.random_seed = 12345
        qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(maxiter=2))  # a qaoa instance of the quadratic program

        
        # ws_qaoa = WarmStartQAOAOptimizer(pre_solver=CobylaOptimizer(),relax_for_pre_solver=True,qaoa=qaoa_mes,epsilon=0.0)
        ws_qaoa = WarmStartQAOAOptimizer(pre_solver=CplexOptimizer(), relax_for_pre_solver=False, qaoa=qaoa_mes,
                                     epsilon=0.0)
        ws_qaoa_result = ws_qaoa.solve(qp)

        # print(ws_qaoa_result.variables_dict.values())

        ans = list(ws_qaoa_result.variables_dict.values())
        # print(dt.datetime.now() - start)

        return ans


    def quantumSolve(self) -> list:
        """
        Linker between all working functions
        :return: list of suitable alternative flights
        """

        total = []
        self.__run()  # we will get the Q A B N G matrices initialised now
        prevAns=[]

        # Runs the algorithm to get maximum number of solutions
        while True:
            ans=self.QAOA_algo()
            if ans == prevAns or ans in total:
                break
            prevAns=ans
            if prevAns == []:
                break
            total.append([self.__postProcess(ans[:self.length])])

        return total


    def __postProcess(self, bitString: list) -> list:
        """
        Convert the output from QAOA_algo() function into a suitable format
        :param bitString:
        :return: list of flights corresponding to the bitString
        """

        path = []
        isStarting=False;isEnding=False # TO check if path is complete (If not complete do no add this path)

        for i in range(len(bitString)):
            if bitString[i]==1:
                added=False
                # Loop to add the flights to the path in order that is like A->B,B->C ... not like B->C,A->B,...
                for idx in range(len(path)):
                    if path[idx]["ArrivalAirport"] == self.lst[i]["DepartureAirport"]:
                        path=path[:idx+1]+[self.lst[i]]+path[idx+1:] # Add this flight before idx th flight
                        added=True
                        break
                    if self.lst[i]["ArrivalAirport"] == path[idx]["DepartureAirport"]:
                        path=path[:idx]+[self.lst[i]]+path[idx:]  # Add this flight after idx th flight
                        added=True
                        break

                if self.lst[i]["DepartureAirport"]==self.flight["DepartureAirport"]:
                    isStarting=True
                    if not added:
                        path=[self.lst[i]]+path # Adds this flight to very start of the path.
                    added=True

                if self.lst[i]["ArrivalAirport"]==self.flight["ArrivalAirport"]:
                    isEnding=True
                if not added:
                    path.append(self.lst[i]) # Adds this flight to last of the path
        if not isEnding or not isStarting:
            return []
        
        # to increase the value of objective function for this path so that next time solver do not consider this flight again.
        for i in range(len(bitString)):
            if bitString[i]==0:
                continue
            for j in range(len(bitString)):
                if bitString[j] == 1 and i!=j:
                    self.MatFlightTime[i,j]=self.highval
                    self.MatFlightTime[j,i]=self.highval                    

        return path


A example 
ImpactedFlights={"INV-ZZ-1875559":"Cancelled","INV-ZZ-1529206":"Delayed","INV-ZZ-2640504":"Cancelled","INV-ZZ-5480266":"Delayed"}

(QuantumSolver("INV-ZZ-1529206",ImpactedFlights,INVFile[0]).solve())  < -- returns the a list of all the alternate solution...

Call this line for every entry of the dictionary:ImpactedFlights  ...

In [ ]:
FullList={"INV-ZZ-5480266":"Cancelled","INV-ZZ-2085988":"Cancelled","INV-ZZ-2640504":"Cancelled","INV-ZZ-5480266":"Delayed"}
iv_id = "INV-ZZ-2085988"
qs = QuantumSolver(inv_id,FullList,data_frames['inventory'])

In [ ]:
qs.solve()